# Technical Indicators

### *Libraries*

In [20]:
# System
import os, time, sys, glob
from pathlib import Path
from datetime import date, datetime, timedelta

In [21]:
# Data
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [22]:
# APIs
import data.marketdata.alpaca as api

In [23]:
# Visualization
import panel as pn
import panel.widgets as pnw

import plotly.express as px
import plotly.graph_objects as go
pn.extension('plotly')
pn.extension()

import holoviews as hv
from holoviews import opts
import hvplot.pandas

import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
# Technical Analysis
from technicals import TechnicalAnalysis

---

### *Data*

#### *Static Data Methods*

Tickers are contained in <code>data/tickers/</code> as <code>.csv</code> files.

The csv structure is
```sql
COLUMNS ('Symbol', 'Name', 'Country', 'IPO Year', 'Sector', 'Industry')
```

In [25]:
# ticker_paths : dict() of str : Path
ticker_paths = ['nasdaq','nyse','amex']

In [26]:
# all tickers
dfs = [pd.read_csv(os.path.join('data','tickers', str(t + '.csv'))).set_index("Symbol") for t in ticker_paths]
for i in range(len(ticker_paths)): dfs[i]['Exchange'] = ticker_paths[i]
tickers_df = pd.concat(dfs, axis='rows')
tickers_df.head()

,Name,Country,IPO Year,Sector,Industry,Exchange
Symbol,,,,,,
AACG,ATA Creativity Global American Depositary Shares,China,NaN,Miscellaneous,Service to the Health Industry,nasdaq
AACQ,Artius Acquisition Inc. Class A Common Stock,United States,2020.0,Finance,Business Services,nasdaq
AACQU,Artius Acquisition Inc. Unit,United States,2020.0,Finance,Business Services,nasdaq
AACQW,Artius Acquisition Inc Warrant,United States,2020.0,Finance,Business Services,nasdaq
AAL,American Airlines Group Inc. Common Stock,United States,NaN,Transportation,Air Freight/Delivery Services,nasdaq


#### *API Data Methods*

##### Alpaca Trade API

In [27]:
ohlcv = api.ohlcv(['FB','AAPL','AMZN','NFLX','GOOG'])
ohlcv.head()

Testing Apaca Trade API key by data type:
ALPACA_API_KEY: <class 'str'>
ALPACA_SECRET_KEY: <class 'str'>


AAPL                                       \
                             open      high      low   close    volume   
time                                                                     
2020-01-02 00:00:00-05:00  296.24  300.6000  295.190  300.58  30589026   
2020-01-03 00:00:00-05:00  297.15  300.5800  296.500  297.38  31609363   
2020-01-06 00:00:00-05:00  293.79  299.9600  292.750  299.78  27487207   
2020-01-07 00:00:00-05:00  299.84  300.9000  297.480  298.27  24353387   
2020-01-08 00:00:00-05:00  297.16  304.4399  297.156  303.15  28984028   

                              AMZN                                          \
                              open       high        low    close   volume   
time                                                                         
2020-01-02 00:00:00-05:00  1874.79  1898.0000  1864.1500  1897.71  3583611   
2020-01-03 00:00:00-05:00  1864.50  1886.1965  1864.5000  1874.93  3293469   
2020-01-06 00:00:00-05:00  1860.00  1903.6900  1860.0000  1903.33  3598872   
2020-01-07 00:00:00-05:00  1904.50  1913.8900  1892.0433  1906.86  3638680   
2020-01-08 00:00:00-05:00  1898.68  1910.9999  1886.4448  1892.09  3032612   

                           ...     GOOG                                        \
                           ...     open     high        low    close   volume   
time                       ...                                                  
2020-01-02 00:00:00-05:00  ...  1341.78  1368.14  1341.5500  1367.73  1259186   
2020-01-03 00:00:00-05:00  ...  1347.86  1372.50  1345.5436  1359.49  1033568   
2020-01-06 00:00:00-05:00  ...  1352.23  1396.50  1350.0000  1393.80  1439587   
2020-01-07 00:00:00-05:00  ...  1397.94  1402.99  1390.3800  1393.33  1296401   
2020-01-08 00:00:00-05:00  ...  1397.95  1411.58  1394.4400  1404.32  1252686   

                             NFLX                                      
                             open      high     low    close   volume  
time                                                                   
2020-01-02 00:00:00-05:00  326.32  329.9800  324.78  329.820  3966616  
2020-01-03 00:00:00-05:00  326.78  329.8599  325.53  325.900  3453110  
2020-01-06 00:00:00-05:00  323.12  336.3600  321.20  335.830  5216762  
2020-01-07 00:00:00-05:00  336.47  336.7000  330.30  330.775  4144924  
2020-01-08 00:00:00-05:00  331.49  342.7000  331.05  339.140  6501692  

[5 rows x 25 columns]

In [9]:
# Test technical analysis class method
stocks = TechnicalAnalysis(ohlcv)
stocks.close(ticker=stocks.tickers()[0])

,close
time,
2020-01-02 00:00:00-05:00,300.58
2020-01-03 00:00:00-05:00,297.38
2020-01-06 00:00:00-05:00,299.78
2020-01-07 00:00:00-05:00,298.27
2020-01-08 00:00:00-05:00,303.15
...,...
2021-05-17 00:00:00-04:00,126.27
2021-05-18 00:00:00-04:00,124.85
2021-05-19 00:00:00-04:00,124.69


---

### *Indicators*

#### RSI - Relative Strenght Index

$$RSI = 100 - \left(\frac{100}{1+Relative\:Strenght}\right)$$

where:
*relative strenght* (*RS*)= *average gain* - *average loss*

In [10]:
# Test technical analysis class method
first_ticker = stocks.tickers()[0]
stocks.close(first_ticker)

,close
time,
2020-01-02 00:00:00-05:00,300.58
2020-01-03 00:00:00-05:00,297.38
2020-01-06 00:00:00-05:00,299.78
2020-01-07 00:00:00-05:00,298.27
2020-01-08 00:00:00-05:00,303.15
...,...
2021-05-17 00:00:00-04:00,126.27
2021-05-18 00:00:00-04:00,124.85
2021-05-19 00:00:00-04:00,124.69


In [11]:
# build plotly display of ohlcv values and rsi
select_ticker = pn.widgets.Select(name='Select', options=list(ohlcv.columns.levels[0]))

# first ticker
first_ticker = stocks.tickers()[0]

# plot close
init_close_plot = stocks.close(ticker=first_ticker)
close_plot = pn.panel(init_close_plot.hvplot(height=400))

# plot rsi
init_rsi_plot = stocks.rsi(ticker=first_ticker)
rsi_plot = pn.panel((init_rsi_plot.hvplot(height=200,
                                           ylim=(0,100)) * hv.HLine(30) * hv.HLine(70)).opts(opts.HLine(color='red', 
                                                                                                        line_width=1)))

# update plot values
@pn.depends(select_ticker, watch=True)
def plot_rsi(select_ticker):
    ticker = select_ticker
    
    update_close_plot = stocks.close(ticker=ticker).hvplot(height=400)         

    update_rsi_plot = (stocks.rsi(ticker=ticker).hvplot(height=200,
                                           ylim=(0,100)) * hv.HLine(30) * hv.HLine(70)).opts(opts.HLine(color='red', 
                                                                                                        line_width=1))     
                   
    close_plot.object = update_close_plot
    
    rsi_plot.object = update_rsi_plot
    
# build panel
rsi_panel = pn.panel(pn.Column("## RSI - Relative Strenght Index", select_ticker, close_plot, rsi_plot))

In [12]:
rsi_panel

Column
    [0] Markdown(str)
    [1] Select(options=['AAPL', 'AMZN', ...], value='AAPL')
    [2] HoloViews(Curve)
    [3] HoloViews(Overlay)

#### Williams %R - William's Percent Range

$$Williams Percent Range=\left(\frac{Highest High-Close}{Highest High-Lowest Low}\right)$$

where:<br>
*Highest High* = Highest price in the lookback period.<br>
*Close* = Most recent closing price.<br>
*Lowest Low* = Lowest price in the lookback period.<br>

In [13]:
# Test technical analysis class method
first_ticker = stocks.tickers()[0]
stocks.williams_range(ticker = first_ticker)

,williams_range
time,
2020-01-02 00:00:00-05:00,0.000000
2020-01-03 00:00:00-05:00,0.000000
2020-01-06 00:00:00-05:00,0.000000
2020-01-07 00:00:00-05:00,0.000000
2020-01-08 00:00:00-05:00,0.000000
...,...
2021-05-17 00:00:00-04:00,-72.874494
2021-05-18 00:00:00-04:00,-82.456140
2021-05-19 00:00:00-04:00,-79.357022


In [14]:
# build plotly display of ohlcv values and williams indicator
select_ticker = pn.widgets.Select(name='Select', options=list(ohlcv.columns.levels[0]))

# first ticker
first_ticker = stocks.tickers()[0]

# plot close
init_close_plot = stocks.close(ticker=first_ticker)
close_plot = pn.panel(init_close_plot.hvplot(height=400))

# plot williams range
init_williams_range_plot = stocks.williams_range(ticker=first_ticker)
williams_range_plot = pn.panel((init_williams_range_plot.hvplot(height=200,
                                           ylim=(-100,0)) * hv.HLine(-50)).opts(opts.HLine(color='red',
                                                                                           line_dash='dashed',
                                                                                           line_width=1)))

# update plot values
@pn.depends(select_ticker, watch=True)
def plot_williams_range(select_ticker):
    ticker = select_ticker
    
    update_close_plot = stocks.close(ticker=ticker).hvplot(height=400)         

    update_williams_range_plot = (stocks.williams_range(ticker=ticker).hvplot(height=200,
                                           ylim=(-100,0)) * hv.HLine(-50)).opts(opts.HLine(color='red',
                                                                                           line_dash='dashed',
                                                                                           line_width=1))
                   
    close_plot.object = update_close_plot
    
    williams_range_plot.object = update_williams_range_plot
    
# build panel
williams_range_panel = pn.panel(pn.Column("## Williams %R - Williams Range", select_ticker, close_plot, williams_range_plot))

In [15]:
williams_range_panel

Column
    [0] Markdown(str)
    [1] Select(options=['AAPL', 'AMZN', ...], value='AAPL')
    [2] HoloViews(Curve)
    [3] HoloViews(Overlay)

#### Aroon Indicator

$$Aroon\:Up= \frac{a_{period} - n_{periods}\:since\:a_{period} max}{a_{period}} * 100$$

$$Aroon\:Down= \frac{a_{period} - n_{periods}\:since\:a_{period} min}{a_{period}} * 100$$

$$Aroon\:Oscillator=Aroon\:Up - Aroon\:Down$$

where:
*where **a<sub>period</sub>** = period of time to be measured*

In [16]:
# Test technical analysis class method
first_ticker = stocks.tickers()[0]
stocks.aroon(ticker = first_ticker)

,aroon_oscillator,aroon_up,aroon_down
2020-01-02 05:00:00,0.0,0.0,0.0
2020-01-03 05:00:00,0.0,0.0,0.0
2020-01-06 05:00:00,0.0,0.0,0.0
2020-01-07 05:00:00,0.0,0.0,0.0
2020-01-08 05:00:00,0.0,0.0,0.0
...,...,...,...
2021-05-17 04:00:00,-68.0,20.0,88.0
2021-05-18 04:00:00,-68.0,16.0,84.0
2021-05-19 04:00:00,-68.0,12.0,80.0
2021-05-20 04:00:00,-68.0,8.0,76.0


In [17]:
# build plotly display of ohlcv values and aroon
select_ticker = pn.widgets.Select(name='Select', options=list(ohlcv.columns.levels[0]))

# first ticker
first_ticker = stocks.tickers()[0]

# plot close
init_close_plot = stocks.close(ticker=first_ticker)
close_plot = pn.panel(init_close_plot.hvplot(height=400))

# plot aroon range plot
init_aroon_range_plot = stocks.aroon(ticker=first_ticker)[['aroon_up','aroon_down']]
aroon_range_plot = pn.panel((init_aroon_range_plot.hvplot(height=200,
                                           ylim=(0,100))))

# plot aroon oscillator plot
init_aroon_oscillator_plot = stocks.aroon(ticker=first_ticker)['aroon_oscillator']
aroon_oscillator_plot = pn.panel((init_aroon_oscillator_plot.hvplot(height=200,
                                           ylim=(-100,100))))

# update plot values
@pn.depends(select_ticker, watch=True)
def plot_aroon(select_ticker):
    ticker = select_ticker
    
    update_close_plot = stocks.close(ticker=ticker).hvplot(height=400)         

    update_aroon_range_plot = (stocks.aroon(ticker=ticker)[['aroon_up','aroon_down']].hvplot(height=200,
                                           ylim=(0,100)))
    
    update_aroon_oscillator_plot = (stocks.aroon(ticker=ticker)['aroon_oscillator'].hvplot(height=200,
                                           ylim=(-100,100)))
                   
    close_plot.object = update_close_plot
    
    aroon_range_plot.object = update_aroon_range_plot
    
    aroon_oscillator_plot.object = update_aroon_oscillator_plot
    
    
# build panel
aroon_panel = pn.panel(pn.Column("## Aroon Indicator", select_ticker, close_plot, aroon_range_plot, aroon_oscillator_plot))

In [18]:
aroon_panel

Column
    [0] Markdown(str)
    [1] Select(options=['AAPL', 'AMZN', ...], value='AAPL')
    [2] HoloViews(Curve)
    [3] HoloViews(NdOverlay)
    [4] HoloViews(Curve)